In [1]:
import os
import subprocess

import re

from string import digits
from itertools import tee
from collections import Counter

from IPython.display import display


In [2]:
path_absolute = 'c:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\'
path_relative = 'FOTOS\\personales\\'
path = path_absolute + path_relative

files_raw = subprocess.Popen(
    'dir ' + path_relative + ' /a:a/b/s', 
    shell=True, stdout=subprocess.PIPE)\
        .stdout.read()\
            .decode().replace('\r', '')
files_raw

\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Yo\\IMG-20160607-WA0000.jpeg\nc:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Yo\\IMG-20160709-WA0007.jpg\nc:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Yo\\IMG-20160803-WA0002.jpg\nc:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Yo\\IMG-20160814-WA0003.jpg\nc:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Yo\\IMG-20160814-WA0004.jpg\nc:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Yo\\IMG-20160825-WA0011.jpg\nc:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Yo\\IMG-20161001-WA0011.jpg\nc:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Yo\\IMG-20161005-WA0000.jpg\nc:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Yo\\IMG-20161015-WA0052.jpg\nc:\\Users\\alanv\\Desktop\\Proyect

In [3]:
re.I, \
re.L, \
re.M, \
re.S, \
re.U, \
re.X, \
re.DEBUG, \
re.A \

(<RegexFlag.IGNORECASE: 2>,
 <RegexFlag.LOCALE: 4>,
 <RegexFlag.MULTILINE: 8>,
 <RegexFlag.DOTALL: 16>,
 <RegexFlag.UNICODE: 32>,
 <RegexFlag.VERBOSE: 64>,
 <RegexFlag.DEBUG: 128>,
 <RegexFlag.ASCII: 256>)

In [4]:
pattern = r'.*\.png|.*\.jpeg|.*\.jpg|.*\.mp4|.*\.wmv|.*\.bmp|.*\.3gp'

expression = re.compile(pattern, 2+32)
files_list = expression.findall(files_raw.replace(path, ''))

print(len(files_list))
files_list[:10]

3157


['A&F\\2015-07-02 12.41.08.jpg',
 'A&F\\20150213_084855.jpg',
 'A&F\\20150213_085142.jpg',
 'A&F\\20150213_085236.jpg',
 'A&F\\20150213_085238.jpg',
 'A&F\\20150213_085314.jpg',
 'A&F\\20150213_085353.jpg',
 'A&F\\20150213_085401.jpg',
 'A&F\\20150224_092916.jpg',
 'A&F\\20150224_092929.jpg']

In [5]:
fold = 'Fernanda\\Portafolio dibujo y Topologia\\'
file = 'IMG_20180101_213649'
extenssion = '.jpg'

name = fold + file + extenssion

try:
    index = files_list.index(name)
except ValueError:
    index = 501
    
print(index)

txt = files_list[index]
txt


2855


'Fernanda\\Portafolio dibujo y Topologia\\IMG_20180101_213649.jpg'

In [6]:
comodin = r'.*'
separation = r'[-_ ]?'

pattern_path = r'(?P<path>.*[\\\w\s]*\\)'

pattern_year = r'(?P<year>20[0-2][0-9])'
pattern_month = r'(?P<month>[01]?[0-9])'
pattern_day = r'(?P<day>[0-3][0-9])'

ext_str = r'\.jpg|\.jpeg|\.png|\.mp4|\.wmv|\.bmp|\.3gp'
# pattern_time_id = f'(?P<time_id>\d+(?={ext_str}))'
pattern_extenssion = f'(?P<ext>{ext_str})'

pattern = ( 
            pattern_path + 
            comodin +
            pattern_year + 
            separation + 
            pattern_month + 
            separation + 
            pattern_day + 
            comodin + 
            # pattern_time_id +
            # comodin +
            pattern_extenssion
           )

reg = re.compile(pattern, 32+2)

print(txt)
print(reg.findall(txt))
print(reg.subn(r'\1\2-\3-\4\5', txt))
print(reg.subn(r'\g<path>\g<year>_\g<month>_\g<day>\g<ext>', txt)[0])

Fernanda\Portafolio dibujo y Topologia\IMG_20180101_213649.jpg
[('Fernanda\\Portafolio dibujo y Topologia\\', '2018', '01', '01', '.jpg')]
('Fernanda\\Portafolio dibujo y Topologia\\2018-01-01.jpg', 1)
Fernanda\Portafolio dibujo y Topologia\2018_01_01.jpg


In [7]:
new_names = []
renamed_count = 0

for file in files_list:
    new_name = reg.subn(r'\g<path>\g<year>\g<month>\g<day>\g<ext>', file)
    new_names.append(new_name[0])
    renamed_count += new_name[1]

print(f'Archivos renombrados:   {renamed_count}')
print('Porcentaje de archivos: {:.2%}'.format( renamed_count / len(files_list) ))

Archivos renombrados:   2497
Porcentaje de archivos: 79.09%


In [8]:
def uniquify(file_list):
    """Make all the items unique by adding a suffix (1, 2, etc).

    `file_list` is mutable list of strings.
    """
    not_unique = [name for name, count in Counter(file_list).items() if count > 1]

    # suffix generator dict - e.g., {'name': <my_gen>, 'zip': <my_gen>}
    suffs = range(1, 100)
    suff_gens = dict(zip(not_unique, tee(suffs, len(not_unique))))  
    
    pattern_path = r'(?P<name>.*(?=\.))'
    pattern_extenssion = r'(?P<ext>\.jpg|\.jpeg|\.png|\.mp4|\.wmv|\.bmp|\.3gp)'
    reg = re.compile(pattern_path + pattern_extenssion)

    for index, name in enumerate(file_list):
        try:
            suffix = str(next(suff_gens[name]))
        except KeyError:
            # name was unique
            continue
        else:
            file_list[index] = reg.sub(r'\g<name>' + f'_{suffix}' + r'\g<ext>', file_list[index])



In [9]:
uniquify(new_names)

range_display = 5
start = index-5 if index-5 >=0 else 0
end = index+5 if index+5 <= len(new_names) else len(new_names)

display(files_list[start:end]) 
display(new_names[start:end])

['Fernanda\\Portafolio dibujo y Topologia\\IMG_20180101_183202.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\IMG_20180101_183226.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\IMG_20180101_183246.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\IMG_20180101_183305.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\IMG_20180101_213628.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\IMG_20180101_213649.jpg',
 'Fernanda\\Yo\\2018-04-28 02.46.19 1.jpg',
 'Fernanda\\Yo\\2018-04-28 02.58.21 2.jpg',
 'Fernanda\\Yo\\2018-05-03 06.44.04 1.jpg',
 'Fernanda\\Yo\\2018-05-03 06.44.06 1.jpg']

['Fernanda\\Portafolio dibujo y Topologia\\20180101_49.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\20180101_50.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\20180101_51.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\20180101_52.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\20180101_53.jpg',
 'Fernanda\\Portafolio dibujo y Topologia\\20180101_54.jpg',
 'Fernanda\\Yo\\20180428_1.jpg',
 'Fernanda\\Yo\\20180428_2.jpg',
 'Fernanda\\Yo\\20180503_1.jpg',
 'Fernanda\\Yo\\20180503_2.jpg']

In [10]:
path + new_names[index], path + files_list[index]

('c:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Portafolio dibujo y Topologia\\20180101_54.jpg',
 'c:\\Users\\alanv\\Desktop\\Proyectos\\img manager\\FOTOS\\personales\\Fernanda\\Portafolio dibujo y Topologia\\IMG_20180101_213649.jpg')

In [11]:
renamed_count = 0
f = open ('errorlog.txt', 'a')

for file, new_file in zip(files_list, new_names):
    try:
        os.rename(path + file, path + new_file)
        renamed_count += 1
    except FileExistsError:
        f.write(f'SRC: {file}\r')
        f.write(f'DST: {new_file}\n\r')

f.close()

print(f'Archivos renombrados:   {renamed_count}')
print('Porcentaje de archivos: {:.2%}'.format( renamed_count / len(files_list) ))

Archivos renombrados:   3155
Porcentaje de archivos: 99.94%
